In [1]:
from info import * 
from preprocess import *

/Users/matthew/opt/anaconda3/envs/social_navigation_task/lib/python3.10/site-packages/openpyxl/worksheet/_read_only.py:79: UserWarning: Unknown extension is not supported and will be removed
  for idx, row in parser.parse():
/Users/matthew/opt/anaconda3/envs/social_navigation_task/lib/python3.10/site-packages/openpyxl/worksheet/_read_only.py:79: UserWarning: Unknown extension is not supported and will be removed
  for idx, row in parser.parse():
/Users/matthew/opt/anaconda3/envs/social_navigation_task/lib/python3.10/site-packages/openpyxl/worksheet/_read_only.py:79: UserWarning: Unknown extension is not supported and will be removed
  for idx, row in parser.parse():


In [ ]:
# # dont count non-responses for averages, etc
# responded  = (behav['button_press'] != 0).values[np.newaxis]
# mask_affil = (behav['dimension'] == 'affil').values
# mask_power = (behav['dimension'] == 'power').values
# resp_mask  =  np.vstack([mask_affil, mask_power]).T * responded

In [15]:
behavior  = pd.read_excel('/Users/matthew/Dropbox/Projects/social_navigation_task/social_navigation_task/../data/example_subject/snt_18001_.xlsx')
 
dim_mask  = np.vstack([(behavior['dimension'] == 'affil').values, (behavior['dimension'] == 'power').values]).T # boolean

responded  = (behavior['button_press'] != 0).values[:,np.newaxis]
resp_mask  = np.multiply(dim_mask, responded) # boolean
decisions  = behavior['decision'].values[:,np.newaxis] * (dim_mask * 1) 


/Users/matthew/opt/anaconda3/envs/social_navigation_task/lib/python3.10/site-packages/openpyxl/worksheet/_read_only.py:79: UserWarning: Unknown extension is not supported and will be removed
  for idx, row in parser.parse():


In [ ]:
# can specify the more exotic options

class ComputeBehavior:

    def __init__(self, file_path, weight_types=None, decision_types=None, out_dir=None):
    
        # annoying warnings that I dont think matter really
        from warnings import simplefilter
        simplefilter(action="ignore", category=pd.errors.PerformanceWarning) # fragmented df
        np.seterr(divide='ignore', invalid='ignore') # division by 0 in some of our operations

        # out directory
        self.out_dir = out_dir
        if self.out_dir is None: 
            self.out_dir = Path(f'{os.getcwd()}/Behavior')
        else:
            if '/Behavior' not in self.out_dir: 
                self.out_dir = Path(f'{self.out_dir}/Behavior')
        if not os.path.exists(self.out_dir):
            print('Creating subdirectory for behavior')
            os.makedirs(self.out_dir)

        # load in data
        self.file_path = Path(file_path)
        self.sub_id    = self.file_path.stem.split('_')[1] # expects a filename like 'snt_subid_*'
        assert utils.is_numeric(self.sub_id), 'Subject id isnt numeric; check that filename has this pattern: "snt_subid*.xlsx"'

        if self.file_path.suffix == '.xlsx':  self.behavior = pd.read_excel(self.file_path, engine='openpyxl')
        elif self.file_path.suffix == '.xls': self.behavior = pd.read_excel(self.file_path)
        elif self.file_path.suffix == '.csv': self.behavior = pd.read_csv(self.file_path)

        # define some options
        self.weight_types = {'':'constant', '_linear-decay':'linear', '_exponential-decay':'exponential'}
        self.wts = list(self.weight_types.keys())
        
        self.decision_types = {'': self.cumulative_coords, '_prev': self.previous_decisions, '_cf': self.counterfactual_decisions}
        self.dts = list(self.decision_types.keys())

    def run_pipeline(self):
        [self.compute_character(c) for c in [1,2,3,4,5,9]]
        self.compute_across_character()
        self.behavior.to_excel(Path(f'{self.out_dir}/snt_{self.sub_id}_behavior.xlsx'), index=False)

    def weight_decisions(self, decisions, decay='constant'):
        ''' 
            different ways to weight the decisions 
        '''
        nt = len(decisions)
        if decay == 'constant':
            weights = np.ones(nt)[:,None] # standard weighting with a constant
        elif decay == 'linear':
            weights = utils.linear_decay(1, 1/nt, nt)[:,None]
        elif decay == 'exponential':
            weights = utils.exponential_decay(1, 1/nt, nt)[:,None]

        return decisions * weights

    def cumulative_coords(self, decisions):
        ''' mainly just to be able to pass a function like for prev decision etc'''
        return decisions.astype(float), np.cumsum(decisions,0).astype(float)

    def previous_decisions(self, decisions):

        # if on each trial the subj represents the last chosen decision/coordinates
        shifted_decisions = _shift_down(decisions, by=1) 
        shifted_coords    = np.cumsum(shifted_decisions, 0)

        return shifted_decisions.astype(float), shifted_coords.astype(float)

    def counterfactual_decisions(self, decisions):
        # - counterfactual wrt each trial
        # -- eg, if on each trial the subj represents the [ultimately] non-chosen decision/coordinates
        coords       = np.cumsum(decisions, 0)
        prev_coords  = coords - decisions # for each trial, undo the decisions by subtracting to get the prev coords
        decisions_cf = -decisions # counterfactual decision
        coords_cf    = prev_coords + decisions_cf # counterfactual coordinates: add the cf decision to the previous coordinate 

        return decisions_cf.astype(float), coords_cf.astype(float)

    def cumulative_mean(self, values, count):
        return np.cumsum(values, 0) / np.cumsum(count, 0)

    def make_3d(self, U, V, ori):
        ''' 
            add 3rd dimension to U & V, as well as to the origin coordinates
            U is vector of interest, z-axis will vary w/ number of interactions
            ori will be subtracted from U, so its z-axis will also vary w/ number of interactions
            V is reference vector, z-axis will remain fixed
        '''

        num = np.arange(1, len(U) + 1)[np.newaxis]
        U   = np.concatenate([U, num], 1) # changes w/ num of interactions
        V   = np.repeat(np.hstack([V, 12])[np.newaxis], len(U), 0) # fixed
        ori = np.concatenate([np.repeat(ori[np.newaxis], len(U), 0), num], 1) # changes w/ num of interactions     

        return U, V, ori                           

    def quadrant_overlap(self, coords):
        ''' 
            the percentage overlap of the decision polygon (made by vertices of coordinates) with each of the 4 2D quadrants 
            sum == 100
        '''

        quadrants = {'1': np.array([[0,0], [0,6],  [6,0],  [6,6]]),   
                     '2': np.array([[0,0], [0,6],  [-6,0], [-6,6]]),
                     '3': np.array([[0,0], [0,-6], [-6,0], [-6,-6]]), 
                     '4': np.array([[0,0], [0,-6], [6,0],  [6,-6]])}

        try: 
            shape   = sp.spatial.ConvexHull(coords)
            polygon = geometry.Polygon(coords[shape.vertices]) 
            overlap = []
            for q, C in quadrants.items(): # overlap w/ quadrants
                Q = geometry.Polygon(C[sp.spatial.ConvexHull(C).vertices])
                overlap.append(polygon.intersection(Q).area/polygon.area) 
            assert np.sum(overlap) == 100, f'Quadrant overlap percentages should sum to 100% but is {np.sum(overlap)}%'
        except:
            overlap = [np.nan,np.nan,np.nan,np.nan]
        
        return overlap

    def compute_character(self, c):
        ''' 
            compute w/in character 
        '''
        ixs   = np.where(self.behavior['char_role_num']==c)[0]
        behav = self.behavior.loc[ixs,:]

        # mask responses
        responded = (behav['button_press'] != 0).values[np.newaxis] # dont count non-responses for averages, etc
        dim_mask  = np.vstack([(behav['dimension'] == 'affil').values, (behav['dimension'] == 'power').values]).T # boolean
        resp_mask = np.multiply(dim_mask, responded) # boolean

        # get decisions into shape=(12,2) [or (3,2) for neutral]
        if 'affil' not in behav.columns: # backward compatability
            decisions = behav['decision'].values[:,np.newaxis] * (dim_mask * 1) 
        else:
            decisions = behav[['affil','power']].values 
        assert decisions.shape == resp_mask.shape, 'decision and response mask arrays are diff shapes'

        #-----------------------------------------
        # weight decisions & calculate coordinates
        #-----------------------------------------

        # loop over diff weighting types 
        for wt, decay in self.weight_types.items():

            ### weight decision updates
            decisions_w = self.weight_decisions(decisions, decay=decay)

            ### compute specific decision types & coordinates
            for dt, dt_func in self.decision_types.items():

                decisions, coords = dt_func(decisions_w)
                self.behavior.loc[ixs, [f'affil{wt}{dt}', f'power{wt}{dt}']]             = decisions
                self.behavior.loc[ixs, [f'affil_coord{wt}{dt}', f'power_coord{wt}{dt}']] = coords

                ### cumulative mean along each dimension [-1,+1] ###
                cum_resp = np.cumsum(resp_mask, 0)
                cum_mean = coords / cum_resp # divide each time point by the response count 
                if c != 9: # neu will be all nans
                    assert np.all(cum_mean[-1,:] == coords[-1,:] / cum_resp[-1,:]), 'cumulative mean is off'
                self.behavior.loc[ixs, [f'affil_mean{wt}{dt}', f'power_mean{wt}{dt}']] = cum_mean

                ### culumative consistency: how far did they go in the direction they were traveling? [0,1] ###
                # 1d consistency = abs value coordinate, scaled by min and max possible coordinate  
                minmax = minmax_coords(decisions)   
                self.behavior.loc[ixs, [f'affil_consistency{wt}{dt}', f'power_consistency{wt}{dt}']] = (np.abs(cum_mean) - minmax[0]) / (minmax[1] - minmax[0])

                # 2d consistency = decision vector length, scaled by min and max possible vector lengths
                min_r = np.array([np.linalg.norm(v) for v in minmax[0]])
                max_r = np.array([np.linalg.norm(v) for v in minmax[1]])
                r     = np.array([np.linalg.norm(v) for v in cum_mean])
                self.behavior.loc[ixs, f'consistency{wt}{dt}'] = (r - min_r) / (max_r - min_r)

                ### multi-dimensional: angles & distances ###

                # directional angles between (ori to poi) and (ori to ref) [optional 3rd dimension]
                # - origin (ori)
                # --- neu: (0, 0, [interaction # (1:12)]) - note that 'origin' moves w/ interactions if in 3d
                # --- pov: (6, 0, [interaction # (1:12)])
                # - reference vector (ref)
                # --- neu: (6, 0, [max interaction (12)])
                # --- pov: (6, 6, [max interaction (12)])
                # - point of interaction vector (poi): (curr. affil coord, power coord, [interaction # (1:12)])
                # to get directional vetctors (poi-ori), (ref-ori)

                ref_frames = {'neu': {'ori': np.array([[0,0]]), 'ref': np.array([[6,0]]), 'dir': False},
                              'pov': {'ori': np.array([[6,0]]), 'ref': np.array([[6,6]]), 'dir': None}} 
                int_num    = np.arange(1, len(behav)+1)[np.newaxis]

                for ndim in np.arange(2, 4):    
                    for ot in ['neu', 'pov']:

                        ## angles between ori-poi & ori-ref
                        poi = coords
                        ref_frame = ref_frames[ot]
                        ref, ori, drn = ref_frame['ref'], ref_frame['ori'], ref_frame['dir']

                        # if 3d, add 3rd dimension
                        if ndim == 3: 
                            poi, ref, ori = self.make_3d(self, poi, ref[0], ori)
                            drn = None # may not be correct for neutral origin... not sure yet

                        # account for origin
                        poi = poi - ori
                        ref = ref - ori

                        # angle between ori-poi & ori-ref
                        self.behavior.loc[ixs, f'{ot}{ndim}d_angle{wt}{dt}'] = utils.calculate_angle(poi, ref, direction=drn) # outputs radians

                        ## distances from ori-poi
                        if ndim == 2: 
                            self.behavior.loc[ixs, f'{ot}_distance{wt}{dt}'] = [np.linalg.norm(v) for v in poi] # l2 norm is euclidean distance from ori

    def compute_across_character(self):
        ''' 
            variables across characters
        '''
        
        for sx in utils.flatten_nested_lists([[f'{wt}{dt}' for dt in self.dts for wt in self.wts]]):
            for t in range(0, 63):

                X = self.behavior.loc[:t, [f'affil_coord{sx}', f'power_coord{sx}', 'char_decision_num']].values

                try: # 3d 
                    vol = sp.spatial.ConvexHull(X).volume
                except: 
                    vol = np.nan
                try: # 2d
                    shape   = sp.spatial.ConvexHull(X[:,0:2]) 
                    perim   = shape.area # perimeter
                    area    = shape.volume  # area when 2D
                except: 
                    perim = np.nan
                    area  = np.nan

                overlap = self.quadrant_overlap(X[:,0:2])

                self.behavior.loc[t, f'perimeter{sx}']  = perim
                self.behavior.loc[t, f'area{sx}']       = area
                self.behavior.loc[t, f'volume{sx}']     = vol
                self.behavior.loc[t, f'Q1_overlap{sx}'] = overlap[0]
                self.behavior.loc[t, f'Q2_overlap{sx}'] = overlap[1]
                self.behavior.loc[t, f'Q3_overlap{sx}'] = overlap[2]
                self.behavior.loc[t, f'Q4_overlap{sx}'] = overlap[3]
                
 

In [2]:
# TODO: concatenate the columns in one go for the df

from pathlib import Path 
import csv
import numpy as np
import pandas as pd
import os
import sys
import scipy.spatial

sys.path.insert(0, '../../../toolbox/toolbox')
from circ_stats import * 
from matrices import *

# should probably import toolbox so circ_stats etc is available...

In [8]:

def by_character(variable):
    ''' return variable name for each character '''
    return [variable + '_' + character_roles[c] for c in range(5)]
    

def chars_df(df, var_name):
    ''' subset a df with the different character values for a variable'''
    return df[[var_name + '_' + cr for cr in character_roles]]


def get_decision_sequence(df):
    ''' grab decisions from a summary df'''
    return df.loc[0, ['decisions_' + "{:02d}".format(t+1) for t in range(63)]]


def repeat_array_cols(X, repeats=2):
    return np.transpose([X] * repeats)


# Log to more detailed xlsx

In [14]:
# file_path = str(Path(f'{pkg_dir}/../data/example_subject/snt_18001.log'))
# experimenter = 'KB'
# parse_log(file_path, experimenter, output_timing=True, out_dir=None)

# Behavior

In [87]:
file_path = Path(f'{pkg_dir}/../data/example_subject/snt_18001.xlsx')
compute_behavior(file_path, out_dir=f'{pkg_dir}/../data/example_subject/')

# for RDVs

In [217]:
from itertools import compress, product, combinations 
def combos(arr, k=2):
    """ 
        arr: np.array to get combos from
        r: num of combos
    """
    return list(combinations(arr, k))

In [242]:
file_path = Path(f'{pkg_dir}/../data/example_subject/snt_18001_behavior.xlsx')
compute_rdvs(file_path, metric='euclidean', out_dir=None)

# summarize all subjs

In [293]:
file_paths = [f'{pkg_dir}/../data/example_subject/snt_18001_behavior.xlsx', f'{pkg_dir}/../data/example_subject/snt_18001_behavior.xlsx']
out_dir=f'{pkg_dir}/preprocessed_behavior/'

summarize_behavior(file_paths, out_dir=out_dir)

/Users/matthew/opt/anaconda3/lib/python3.9/site-packages/openpyxl/worksheet/_read_only.py:79: UserWarning: Unknown extension is not supported and will be removed
  for idx, row in parser.parse():


/Users/matthew/opt/anaconda3/lib/python3.9/site-packages/openpyxl/worksheet/_read_only.py:79: UserWarning: Unknown extension is not supported and will be removed
  for idx, row in parser.parse():
